# Peak finding in a stellar image

## Which data are used in this notebook?

The image in this notebook is of the field of the star [TIC 125489084](https://exofop.ipac.caltech.edu/tess/target.php?id=125489084) on a night when the moon was nearly full. The moonlight caused a smooth gradient across the background of the image.

As discussed in NOTEBOOK ABOUT BACKGROUND SUBTRACTION, the best way to remove the background in this case is to use photutils to construct a 2D model of the background. 

As we will see in a moment, [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) does a good job identifying most of the sources in the image, since all of the sources are stars.

In [ ]:
import warnings

from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, gaussian_sigma_to_fwhm, SigmaClip
from astropy.table import QTable
from astropy.visualization import simple_norm, SqrtStretch
from astropy.visualization.mpl_normalize import AsymmetricPercentileInterval, ImageNormalize

import matplotlib.pyplot as plt
import numpy as np

from photutils.aperture import CircularAperture, EllipticalAperture
from photutils.background import Background2D, MeanBackground
from photutils.centroids import centroid_2dg
from photutils.datasets import make_100gaussians_image, make_gaussian_sources_image
from photutils.detection import find_peaks, DAOStarFinder

plt.style.use('../photutils_notebook_style.mplstyle')

To begin, let's create the image and subtract the background. Recall that this image contains a mix of star-like object and more extended sources.

In [ ]:
ccd_image = CCDData.read('TIC_125489084.01-S001-R055-C001-ip.fit.bz2')

sigma_clip = SigmaClip(sigma=3., maxiters=5)
bkg_estimator = MeanBackground()
bkg = Background2D(ccd_image, box_size=200, filter_size=(9, 9), mask=ccd_image.mask,
                   sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
# Calculate the 2D background subtraction, maintaining metadata, unit, and mask
ccd_2d_bkgdsub = ccd_image.subtract(bkg.background)

data_subtracted = ccd_2d_bkgdsub.data

In [ ]:
mean, med, std = sigma_clipped_stats(data_subtracted, sigma=3.0, maxiters=5)

## Source Detection with peak finding

For more general source detection cases that do not require comparison with models, `photutils` offers the [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) function. 

This function simply finds sources by identifying local maxima above a given threshold and separated by a given distance, rather than trying to fit data to a given model. Unlike the previous detection algorithms, [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) does not necessarily calculate objects' centroids. Unless the `centroid_func` argument is passed a function like `photutils.centroids.centroid_2dg` that can handle source position centroiding, `find_peaks` will return just the integer value of the peak pixel for each source.

This algorithm is particularly useful for identifying non-stellar sources or heavily distorted sources in image data.

The centroid-finding, which we do here only so that we can compare positions found by this method to positions found by the star-finding methods, can generate many warnings when an attempt is made to fit a 2D Gaussian to the sources. The warnings are suppressed below to avoid cluttering the output.

Let's see how it does:

In [ ]:
with warnings.catch_warnings(action="ignore"):
    sources_findpeaks = find_peaks(data_subtracted,
                                   threshold=5. * std, box_size=29,
                                   centroid_func=centroid_2dg)
print(sources_findpeaks)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the 2D-subtracted data
norm = ImageNormalize(ccd_2d_bkgdsub, interval=AsymmetricPercentileInterval(30, 99.5))

fitsplot = ax1.imshow(data_subtracted, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=30, marker='o',
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')
ax1.set_title("find_peaks, no masking")


The column on the left side of this CCD is not actually exposed to light, so [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks) finds a bunch of spurious sources. Recall that we can provide a mask to the source finder. As we did in [XX BROKEN LINK FEDER DAO](), we exclude the region within 50 pixels of the edge.

In [ ]:
# Create a mask array
mask = np.zeros_like(ccd_2d_bkgdsub, dtype=bool)

# Set the mask for a border of 50 pixels around the image
mask[:50, :] = mask[-50:, :] = mask[:, :50] = mask[:, -50:] = 1.0

with warnings.catch_warnings(action="ignore"):
    sources_findpeaks = find_peaks(data_subtracted, mask=mask,
                                   threshold=5. * std, box_size=29,
                                   centroid_func=centroid_2dg)


In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the 2D-subtracted data
norm = ImageNormalize(ccd_2d_bkgdsub, interval=AsymmetricPercentileInterval(30, 99.5))

fitsplot = ax1.imshow(data_subtracted, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=30, marker='o',
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

ax1.set_title("find_peaks, image edges masked");

## Comparing Detection Methods

Let's compare how peak finding compares to using [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) on this image.


In [ ]:
# Create the star finder
fwhm = 6.2
daofinder = DAOStarFinder(threshold=5 * std, fwhm=2 * fwhm)
sources_dao = daofinder(ccd_2d_bkgdsub.data, mask=mask)

In [ ]:
print(f'''DAOStarFinder: {len(sources_dao)} sources
find_peaks: {len(sources_findpeaks)} sources''') 

And just for fun, let's plot these matching sources. (The colors chosen to represent different sets are from [Paul Tol's guide for accessible color schemes](https://personal.sron.nl/~pault/).)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(data_subtracted, norm=norm, cmap='Greys', origin='lower')

marker_size = 60
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=marker_size, marker='s',
            lw=1, alpha=1, facecolor='None', edgecolor='r', label='Found by find_peaks')
ax1.scatter(sources_dao['xcentroid'], sources_dao['ycentroid'], s=2 * marker_size, marker='D',
            lw=1, alpha=1, facecolor='None', edgecolor='#0077BB', label='Found by DAOfind')

# Add legend
ax1.legend(ncol=2, loc='lower center', bbox_to_anchor=(0.5, -0.20))


ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Sources Found by Different Methods');

*Remember that you can double-click on the plot to zoom in and look around!*

## Summary

In this case, [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) performs better than [`find_peaks`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks). This makes sense, since [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) is tuned for finding stars, and all of the sources in this image are stars.